## API Academia Canaria de la Langua

Las API de la ACL se pueden consultar sin necesidad de autentificación. Todas las operaciones
son con `GET` y devuelven como resultado un JSON. El resultado _siempre_ es un diccionario. Este diccionario tendrá _siempre_ un campo `status`.

- Si todo ha ido bien, el campo `status` tendrá como contenido `ok`, y habrá
  otro campo en el diccionario con el nombre `result` que tendrá el resultado
  de la api, que podrá ser un número, diccionario, lista, o cualquier otro
  resultado válido en JSON.

- Si se ha producido un error, en la clave `status` encontraremos el valor `error`, __no__ habrá
  un campo `result`, pero si habrá un campo `message` con más información acerca del error
  peoducido.

Esto son los campos incluidos por defecto, pero dependiendo de la llamada, puede
que haya más claves con meta-informnación sobre el resultado, algunos
de los más habituales son:

- `timestamp`: marca temporal que indica el momento en que se realizó la respuesta. Como
  JSON no tiene un tipo especifico para fechas/timestamps, el valor es una
  cadena de texto en formato [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601). Si
  en el resultado hubieran también fechas o timestamp, se codificarían igual.

- `name`: Nombre de la API que se ha invocado.

- `length`: Si el dato devuelto en el campo `result` tienen el concepto 
  de longitud, tendría ese valor. Por ejemplo, si el resultado es una lista
  o un diccionario, en este campo vendrían el número de elementos. Si el
  resultado es una string de texto, la longitud de la misma. Si es un entero,
  no vendrá este campo, ya que no tiene sentido en ese caso.

### Versión de la API

La llamada API más sencilla de todas es la que nos permite comprobar la versión de la API activa.

Para consultarla, solo hay que hacer un GET a la ruta `/api/version/` en el website de la Academia, `wwww.academiacanarialengua.org`. El siguiente código realiza una consulta a esa API usando la librería [requests](http://docs.python-requests.org/en/master/) de Python.

In [1]:
import requests
host = 'www.academiacanarialengua.org'
url = 'http://{host}/api/version/'.format(host=host)

data = requests.get(url)
print(data.json())

{'status': 'ok', 'result': '0.1', 'timestamp': '2018-06-29 22:19:33.400743', 'length': 3, 'name': 'version'}


Ejemplo de consulta usando el navegador:

- [http://www.academiacanarialengua.org/api/version/](http://www.academiacanarialengua.org/api/version/)

### la API de búsqueda

La api de búsqueda está en la ruta `api/search/<raiz>`, de forma que podemos buscar todas las palabras que _empiezan_
por una serie de letras, por ejemplo, para buscar las palabras que empiezan por `agu`, la URL a usar
sería [http://www.academiacanarialengua.org/api/search/agu](http://www.academiacanarialengua.org/api/search/agu). El siguiente ejemplo lista las palabras encontradas usando esa misma llamada:

In [2]:
import requests
host = 'www.academiacanarialengua.org'
term = 'agu'
url = 'http://{host}/api/search/{term}'.format(host=host, term=term)

r = requests.get(url)
data = r.json()
print(data['status'])
for row in data['result']:
    print(row['id_entrada'], row['palabra'])

ok
525 agua
526 aguacatero
527 aguachento
528 aguachirre
529 aguadija
530 aguadura
531 aguafeble
532 aguaidil
533 aguaje
534 aguantamicuna
535 aguantar
536 aguapata
537 aguasperarse
538 aguaviva
539 agüero
540 aguiciar
541 águila
542 aguililla
543 agüimense
544 agujerada
545 agujeta
546 agujilla
547 agulense


### La API de consulta de detalles de una entrada

A partir del identificador de cada entrada, que es el número `id_entrada` devuelto por la API
de búsqueda, podemos obrtener todos los datos de esa entrada, especialmente la lista de acepciones
del término. La api esta localizada en `api/word/<id. entrada>`. 

Por ejemplo, podemos obtener los detalles de la entrada _aguaviva_ (Id. 538) con la siguiente
llamada:

- [http://www.academiacanarialengua.org/api/word/538](http://www.academiacanarialengua.org/api/word/538)

In [3]:
from IPython.core.display import display, HTML
import requests

host = 'www.academiacanarialengua.org'
id_word = 538
url = 'http://{host}/api/word/{pk}'.format(host=host, pk=id_word)

data = requests.get(url).json()
word = data['result']
for k in word:
    if k != 'acepciones':  # Veremos las acepciones más adelante
        print(k, data['result'][k])

id_entrada 538
version 0
destacada True
palabra aguaviva
slug aguaviva
drae False
variante 
revisada True
edicion 1


#### Descripción de los campos

Los campos de una palabra son los siguientes:

- **`id_entrada`**: La clave primaria e identificador único de cada entrada en el 
  diccionario.

- **`edicion`**: Se refiere a la edición en papel. Estos contenidos son los que están publicados
    en la versión actual, que es la 1. Hay una versión en desarrollo en la que todas
    las entradas tendrán en este valor un 2, pero por el momento no es pública
    
- **`version`**: La versión de la entrada. Hay alguna entradas que corresponden 
  a palabras que se escriben igual pero resultan ser entradas diferentes. Véase por 
  ejemplo [baifa](http://www.academiacanarialengua.org/palabra/baifa/), que aparece 
  dos veces, con id. 5112, version 1 e id. 860, version 2. A pesar de que se escriben
  igual, son dos entradas distintas en el diccionario, porque
  la versión 1 es un sustantivo y la 2 es un adjetivo. Si `version` es cero significa que
  no hay ninguna otra entrada que se escriba igual que esta. Si hay duplicados, `version`
  valdra $1$, $2$,...

- **`palabra`**: El texto de la palabra, siempre en minúscula.

- **`slug`**: Se usa para componer el URL de la palabra en la web. Normalemente coincide
  con la palabra, pero a veces es una versión más sencilla, sin acentos, eñes, etc...
  para que las direcciones se puedan escribir usando cualquier teclado. Por ejemplo
  [abejón](http://www.academiacanarialengua.org/palabra/abejon) tiene como 
  slug `abejon`. Observese que el campo `slug` es la última parte de la ruta. 
  
  La combinación de `slug` y `version` forman
  un par de valores único para cada palabra en el diccionario y son, por tanto, una 
  clave candidata u opcional.

- **`variante`**: Se usa para palabras que tienen una variacíón por género o por otras
  causas, véase por 
  ejemplo [abaifado/da](http://www.academiacanarialengua.org/palabra/abaifado). El valor
  del campo `variante` en este caso es `da`.
  
- **`revisada`**: Si está a verdadero, es que la entrada está revisada. Todas las entradas
  en el diccionario actual están revisadas, es decir, todas tienen este campo a `True`.
  
- **`drae`**: Indicaría si esta palara está recogida en DRAE o no. No es un dato fiable.
  
- **`destacada`**: Este campo es un booleano que indica si la palabra debe aparecer un la
  pagina principal de la web de la ACL. Asi evitamos que aparezcan palabras con poco
  contenido, sinónimos, etc...
  


### Las acepciones

Además de lo anterior, la palabra tiene una lista de acepciones. Cada una de las
acepciones tiene los siguientes campos.

- **`id_acepcion`**: El identificador único y clave primaria de la acepción

- **`orden`**: El orden en que se debe presentar cada acepción

- **`formato`**: Siempre debbería ser `html`

- **`texto`**: El lexto de la acepción.

- **`etiquetas`**: Una lista de las etiquetas asociadas a la acepción. Cada
  etiqueta se compone de 5 campos:
  
  - **`id_etiqueta`**: El identificador de la etiqeuta
  
  - **`descripcion`**: La descripción textual
  
  - **`etiqueta`**: La marca usada para indicar esa etiqueta, p.e. `m` para
    sustantivo masculino.
    
  - **`orden`**: El orden en que se deben mostrar las etiquetas.  
    
  - **`grupo`**: Una letra que indica el grupo al que corresponde la etiqueta. Hay tres grupos
    definidos en este diccionario: Gramatical (`G`), Diatópicas (`L`) y Otras (`X`).




Veamos un ejemplo donde mostramos las acepciones de la palabra _aguaviva_ (Id. 538):

In [4]:
from IPython.core.display import display, HTML
import requests

host = 'www.academiacanarialengua.org'
id_word = 538
url = 'http://{host}/api/word/{pk}'.format(host=host, pk=id_word)

data = requests.get(url).json()
word = data['result']
display(HTML('<h4>Acepciones</h4>'))
display(HTML('<ul>'))
for acepcion in word['acepciones']:
    etiquetas = ','.join([
        etiqueta['etiqueta']
        for etiqueta in acepcion['etiquetas']
        ])
    display(HTML('<li><b>{}</b> <tt>{}</tt> {}</li>'.format(
        acepcion['orden'],
        etiquetas,
        acepcion['texto'],
    )))
    
display(HTML('</ul>'))

#### Acepciones como ejemplos ⭐

Algunas acepciones son en realidad ejemplos de uso. Se pueden distinguir porque
el texto empieza siempre con la cadena de texto `//`. Véase como ejemplo
la entrada **Aceite (id. 487)**.

 - [Aceite (web)](http://www.academiacanarialengua.org/palabra/aceite/)
 - [Aceite (api)](http://www.academiacanarialengua.org/api/word/487)

#### Acepciones que citan otras entradas ⭐

Algunas acepciones usan o _citan_ en su explicación otra entrada diferente del 
diccionario. En ese caso, en la acepción se incluye un campo
`cita` que apunta a la entrada usada. Por ejemplo, la entrada **abochacharse (id. 461)**, 
en su primera acepción, referencia a la entrada **hincharse (id. 2763)**.
El campo cita incluye los valores:

 - **`palabra`**. La palabra usada
 - **`id_entrada`**: El identificador numérico de la entrada
 
Vease como ejemplo:

 - [abochacharse (web)](http://www.academiacanarialengua.org/palabra/abochacharse/)
 - [abochacharse (api)](http://www.academiacanarialengua.org/api/word/461)
 
 **Nota**: En la versión actual de la web, no se usa esta información, pero la aplicación
 móvil podría usarla, si lo consideráis pertinente.

#### Acepciones con notas ⭐

Algunas acepciones incluyen notas, ya sea sobre pronunciación o haciendo
referencia a alguna foto que está asociada a otra entrada. en esos casos,
la acepcion incluirá un campo `nota` adicional. Véase como ejemplo
la entrada **caracolear (id. 1322)**, que tiene una nota sobre pronunciación:

 - [caracolear (web)](http://www.academiacanarialengua.org/palabra/caracolear/)
 - [caracolear (api)](http://www.academiacanarialengua.org/api/word/1322)
 
O la entrada **tabaquero (id. 4508)**, que incluye una nota haciendo referencia
a la foto de otra entrada, **camello (id. 1254)**. En este caso, los campos `foto` y `thumbnail` estarán incluidos con la información de dicha foto (Véase la
sección: _Entradas con fotos_).

 - [tabaquero (web)](http://www.academiacanarialengua.org/palabra/tabaquero/)
 - [tabaquero (aip)](http://www.academiacanarialengua.org/api/word/4508)


#### La entrada que más acepciones tiene (Papa)⭐

He pensado que podría seros interesante saber que entrada tiene ahora
mismo más acepciones, para probar la interfaz, tiempos de respuesta, etc...
Se trata de **papa (id. 3582)**, con $23$ acepciones (y foto):

 - [papa (web)](http://www.academiacanarialengua.org/palabra/papa/)
 - [papa (api)](http://www.academiacanarialengua.org/api/word/3582)



### Entradas con fotos ⭐

las entradas que tengan foto incluirán doc campos más, `foto` y `thumbnail`. el campo `foto` incluye la siguiente información:
    
- **`url`**: La URl relativa de la foto.
- **`peso`**: El tamaño en bytes del fichero
- **`ancho`**: Ancho en pixels de la imagen
- **`alto`**: Alto en pixels de la imagen
- **`leyenda`**: Leyenda para incluir al pie de la foto

El campo `thumbnail` es una versión reducida en tamaño de la misma foto. Está
garantizado que su tamaño siempre será, como máximo, $320x200$ o $200x320$ (según
el formato sea apaisado o retrato). Los campos son los mismo que en la foto, pero
se omite la leyenda.

Vease como siguiente ejemplo la entrada **Abade (Id. 440)**:

 - [abade (web)](http://www.academiacanarialengua.org/palabra/abade/)
 - [abade (api)](http://www.academiacanarialengua.org/api/word/440)

#### Api de busqueda que se podrían hacer facilmente

No están hechas, pero estas API se podrían hacer muy facilmente:
    
- Búsqueda de palabras que **contengan** un determinado fragmento (no solo las que _empiezan por_)

- Búsqueda de palabras que incluya el texto de las acepciones.
